## 以下会使用到的API有：
- 百度语音的语音识别api
- 高德地图的地理编码api
- 百度图像识别的人脸对比api

## 1、语音识别API的输入与输出示例

 ### 解释：输入学号语音 → 通过api处理 → 输出学号的文字 → 调出对应用户的签到人照片（用于第三部的人脸对比）
 - 代码中用的为wav档的录音文件，在同目录下有放置

In [2]:
from aip import AipSpeech

""" 你的 APPID AK SK """
APP_ID = '***'
API_KEY = '***'
SECRET_KEY = '***'

client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)

# 读取文件
def get_file_content(filePath):
        with open(filePath, 'rb') as fp:
            return fp.read()

# 识别本地文件
rtn = client.asr(get_file_content('test3.wav'), 'wav', 16000, {
        'dev_pid': 1536,
})

print(rtn['result'][0])

2222


## 2、高德API的输出输出示例

### 解释：输入上课地点 → 通过高德api处理 → 输出上课地点的经纬度 → 与签到人定位的地点进行对比，最终确定签到人是否在上课地点附近

In [1]:
import requests

#申请的key
key='***'

#传入地址，返回对应地址的经纬度信息
def address(address):
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(key,address)
    data=requests.get(url)
    contest=data.json()
    contest=contest['geocodes'][0]['location']
    return contest

if __name__ == '__main__':
    print(address("广东省中山大学南方学院教学楼4号"))

113.392782,22.517645


## 3、人脸对比API的输入与输出示例

### 解释：输入签到人现场拍的照片和第一步中学号提取出来的照片 → 通过人脸对比api → 输出相似度的多少 → 最终通过相似度来判断是否为签到人。
- 代码中用的为张一山的证件照和生活照作为对比，照片材料有放在pic的文件夹中。

In [41]:
from aip import AipFace
import base64

APP_ID = '***'
API_KEY = '***'
SECRET_KEY = '***'
client = AipFace(APP_ID, API_KEY, SECRET_KEY)

IMAGE_TYPE='BASE64'

f1 = open('./pic/live.jpg','rb')
f2 = open('./pic/IDcard.jpg','rb')
#参数image：图像base64编码 分别base64编码后的2张图片数据
img1 = base64.b64encode(f1.read())
img2 = base64.b64encode(f2.read())
image_1 = str(img1,'utf-8')
image_2 = str(img2,'utf-8')

ptr = client.match([
       {
                'image': image_1,
                'image_type': 'BASE64',
       },
        {
                'image': image_2,
                'image_type': 'BASE64',
        }
])
ptr = ptr['result']
if ptr['score']<= 80:
    print('检测结果：非签到者本人',"得分为:",ptr['score'])
else:
    print('检测结果：为签到者本人。',"得分为:",ptr['score'])

检测结果：为签到者本人。 得分为: 89.79454041
